**마찬가지로 베이지안 최적화로 파라미터를 찾으려 했으나 시간 관계상 랜덤화 탐색 기법으로 찾음**.
추후 프로젝트 이후 시간이 충분하다면 그리드 탐색 기법으로 모든 경우 탐색해볼 예정

랜덤화 기법으로 파라미터 탐색 후 그나마 가장 괜찮게 판단된 모델(만약 거의 비슷하거나 차이가 유의하지 않을 정도로 판단된다면 베이스 모델로 하기 $\rightarrow$ 이상치는 잘못 기입한 값이 아닌 이상 자연적인 현상으로 처리하지 않는 것이 가장 현상을 잘 반영할 것이기 때문).

# 0. 패키지 불러오기

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display as dp
import warnings
import matplotlib
import matplotlib.pyplot as plt

# 스케일링 => KNN은 거리 기반이므로 이상치에 민감하니 StandardScaler 사용
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

warnings.filterwarnings('ignore')
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [3]:
# 검증과 테스트 데이터셋은 피처 스케일링되어 사용되므로 먼저 정의하였다.
validation =pd.read_csv('../data/validation_encoding.csv', index_col=0)
test =pd.read_csv('../data/test_encoding.csv', index_col=0)

X_val, y_val = validation.iloc[:,:-1], validation.iloc[:,-1] # 그대로 사용됨.
X_test, y_test = test.iloc[:,:-1], test.iloc[:,-1] # 그대로 사용됨.

dp(validation,test)
dp(X_val, y_val.to_frame() , X_test, y_test.to_frame())

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3,TARGET
73378,3,4.0,10,750,22.0,3.711225,0.000000,0.0,0.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,2
71524,8,0.0,10,329,12.0,2.734892,0.000000,0.0,0.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,1
38781,3,0.0,4,649,10.0,1.294901,0.000000,0.0,0.0,335.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
35212,10,5.0,6,628,20.0,1.337039,0.000000,0.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
56852,8,2.0,6,1846,36.0,2.158072,415.666667,0.0,0.0,205.0,...,1.0,0.0,0.0,0.0,0.0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23050,9,6.0,6,2148,38.0,2.184994,3.125000,0.0,0.0,220.0,...,1.0,0.0,0.0,0.0,0.0,1,0,0,0,0
36543,4,6.0,1,1827,51.0,2.204480,0.000000,0.0,0.0,215.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0
70132,11,4.0,2,1349,36.0,1.873360,1.142857,0.0,0.0,285.0,...,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0
58097,9,0.0,12,329,12.0,3.002181,19.625000,0.0,0.0,165.0,...,1.0,0.0,0.0,0.0,0.0,1,1,0,1,0


,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3,TARGET
12635,9,3.0,10,1050,32.0,1.254874,0.000000,0.0,0.000000,105.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
22195,5,5.0,10,1447,41.0,1.494139,5.142857,0.0,0.000000,220.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
42119,11,6.0,4,1348,6.0,2.063934,0.000000,0.0,0.000000,30.0,...,0.0,0.0,0.0,1.0,0.0,1,0,0,0,2
34954,1,2.0,1,746,21.0,1.746713,22.000000,35.0,48.333333,210.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
65122,4,3.0,2,426,13.0,2.079781,0.555556,0.0,0.000000,220.0,...,1.0,0.0,0.0,0.0,0.0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41734,11,5.0,9,936,28.0,0.650350,0.000000,0.0,0.000000,320.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
16135,6,1.0,8,1938,52.0,2.386494,24.250000,0.0,0.000000,200.0,...,1.0,0.0,0.0,0.0,0.0,0,0,1,0,1
65530,1,2.0,9,1148,29.0,0.446058,28.875000,34.0,144.000000,345.0,...,0.0,0.0,0.0,1.0,0.0,0,0,1,1,1
80594,1,5.0,6,36,46.0,2.517878,23.000000,15.8,22.666667,310.0,...,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0


,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3
73378,3,4.0,10,750,22.0,3.711225,0.000000,0.0,0.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
71524,8,0.0,10,329,12.0,2.734892,0.000000,0.0,0.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
38781,3,0.0,4,649,10.0,1.294901,0.000000,0.0,0.0,335.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
35212,10,5.0,6,628,20.0,1.337039,0.000000,0.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
56852,8,2.0,6,1846,36.0,2.158072,415.666667,0.0,0.0,205.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23050,9,6.0,6,2148,38.0,2.184994,3.125000,0.0,0.0,220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0,0
36543,4,6.0,1,1827,51.0,2.204480,0.000000,0.0,0.0,215.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
70132,11,4.0,2,1349,36.0,1.873360,1.142857,0.0,0.0,285.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0
58097,9,0.0,12,329,12.0,3.002181,19.625000,0.0,0.0,165.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1,1,0,1


,TARGET
73378,2
71524,1
38781,0
35212,0
56852,1
...,...
23050,0
36543,0
70132,0
58097,0


,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3
12635,9,3.0,10,1050,32.0,1.254874,0.000000,0.0,0.000000,105.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
22195,5,5.0,10,1447,41.0,1.494139,5.142857,0.0,0.000000,220.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
42119,11,6.0,4,1348,6.0,2.063934,0.000000,0.0,0.000000,30.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1,0,0,0
34954,1,2.0,1,746,21.0,1.746713,22.000000,35.0,48.333333,210.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
65122,4,3.0,2,426,13.0,2.079781,0.555556,0.0,0.000000,220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41734,11,5.0,9,936,28.0,0.650350,0.000000,0.0,0.000000,320.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
16135,6,1.0,8,1938,52.0,2.386494,24.250000,0.0,0.000000,200.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,0,1,0
65530,1,2.0,9,1148,29.0,0.446058,28.875000,34.0,144.000000,345.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0,0,1,1
80594,1,5.0,6,36,46.0,2.517878,23.000000,15.8,22.666667,310.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0


,TARGET
12635,1
22195,1
42119,2
34954,0
65122,0
...,...
41734,1
16135,1
65530,1
80594,0


In [4]:
# 피처 스케일링에 사용될 scaler
scaler = StandardScaler()

# base model의 학습 데이터
base_train = pd.read_csv('../data/base_train_encoding.csv', index_col=0)
X_train_base, y_train_base = base_train.iloc[:,:-1], base_train.iloc[:,-1]

X_train_base_scaled = scaler.fit_transform(X_train_base)

# 어떤 모델이건 사용될 검증/테스트 데이터셋 => 학습데이터셋이 다를 때마다 fit_transform새로 해서 transform해야함.
X_val_base_scaled = scaler.transform(X_val)
X_test_base_scaled = scaler.transform(X_test)

X_train_base_scaled.shape # pd.DataFrame(X_train_base_scaled, index = X_train_base.index, columns = X_train_base.columns)

(50643, 21)

___

# 1. 베이스 모델
- 이상치 처리는 하지 않고 피처 스케일링만 한 데이터셋으로 학습 후 확인 

랜덤화 사용하기 베이지안 안됨

In [5]:
from time import time
start = time()

mlp = MLPClassifier(solver='adam', activation='relu')

# 적절한 hidden_layer_size와 max_iter값을 위한 과정
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV  # GridSearchCV는 모든 경우를 테스트하기 때문에 속도가 느림. 그러나 RandomizedSearchCV는 일부만 조합해서 최적의 값을 못 찾을지도
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer # 베이지안 최적화 수행 시 파라미터 공간이 어떤 공간인지를 정의하기 위해 필요
from scipy.stats import uniform
# 은닉층 개수는 2개까지, 은닉층의 노드 개수는 피처 개수 이하로 설정.
# params = {
#     'hidden_layer_sizes' : Categorical([(10,), (20,), (10,10), (20,20)]),
#     'alpha' : Real(0.0001, 0.05, 'uniform'),
#     'learning_rate' : Categorical(['constant']),
#     'max_iter' : Integer(1000,2000)
# }
params = {
    'hidden_layer_sizes' : [(10,), (20,)],
    'alpha' : uniform(loc=0.001, scale=0.01),
    'learning_rate' : ['constant'],
    'max_iter' : range(1000,2000)
}
random_search = RandomizedSearchCV(mlp, params, cv = 5) # 5겹 교차 검증 보통 사용하는 값으로 10도 있음.
random_search.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')
params_base = random_search.best_params_
print(params_base)

베이스 모델의 파라미터 찾기 : 492.4078550338745
{'alpha': 0.0060287190137804075, 'hidden_layer_sizes': (10,), 'learning_rate': 'constant', 'max_iter': 1754}


In [6]:
# mlp 모델 학습
start = time()

mlp_base = MLPClassifier(hidden_layer_sizes = params_base['hidden_layer_sizes'],
                         alpha = params_base['alpha'],
                         learning_rate = params_base['learning_rate'],
                         max_iter = params_base['max_iter'])
mlp_base.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')

# 검증 데이터를 이용한 예측
y_val_pred = mlp_base.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 9.177528619766235
              precision    recall  f1-score   support

           0       0.56      0.73      0.63      7276
           1       0.56      0.37      0.44      5052
           2       0.44      0.39      0.41      4553

    accuracy                           0.53     16881
   macro avg       0.52      0.50      0.50     16881
weighted avg       0.53      0.53      0.52     16881

[[5321  793 1162]
 [2123 1857 1072]
 [2108  682 1763]]
0.5296487174930395
0.5296487174930395


___

# 2. IQR_삭제 데이터셋
- 울타리 바깥 범위의 데이터를 삭제한 데이터셋으로 학습 후 확인

In [7]:
IQR_delete = pd.read_csv('../data/IQR_delete_encoding.csv',index_col=0)
X_train_IQR_delete, y_train_IQR_delete = IQR_delete.iloc[:,:-1], IQR_delete.iloc[:,-1]

scaler = StandardScaler()
X_train_IQR_delete_scaled = scaler.fit_transform(X_train_IQR_delete)
X_val_IQR_delete_scaled = scaler.transform(X_val)
X_test_IQR_delete_scaled = scaler.transform(X_test)

In [8]:
start = time()

mlp = MLPClassifier(solver='adam', activation='relu')

# 적절한 hidden_layer_size와 max_iter값을 위한 과정
# 은닉층 개수는 2개까지, 은닉층의 노드 개수는 피처 개수 이하로 설정.
# params = {
#     'hidden_layer_sizes' : Categorical([(10,), (20,), (10,10), (20,20)]),
#     'alpha' : Real(0.0001, 0.05, 'uniform'),
#     'learning_rate' : Categorical(['constant']),
#     'max_iter' : Integer(1000,2000)
# }
params = {
    'hidden_layer_sizes' : [(10,), (20,)],
    'alpha' : uniform(loc=0.001, scale=0.01),
    'learning_rate' : ['constant'],
    'max_iter' : range(1000,2000)
}
random_search = RandomizedSearchCV(mlp, params, cv = 5) # 5겹 교차 검증 보통 사용하는 값으로 10도 있음.
random_search.fit(X_train_IQR_delete_scaled, y_train_IQR_delete)
print(f'IQR_삭제 모델 파라미터 찾기 : {time()-start}')
params_IQR_delete = random_search.best_params_
print(params_IQR_delete)

IQR_삭제 모델 파라미터 찾기 : 378.26549077033997
{'alpha': 0.0019745792578906244, 'hidden_layer_sizes': (10,), 'learning_rate': 'constant', 'max_iter': 1655}


In [9]:
# mlp 모델 학습
start = time()

mlp_IQR_delete = MLPClassifier(hidden_layer_sizes = params_IQR_delete['hidden_layer_sizes'],
                         alpha = params_IQR_delete['alpha'],
                         learning_rate = params_IQR_delete['learning_rate'],
                         max_iter = params_IQR_delete['max_iter'])

mlp_IQR_delete.fit(X_train_IQR_delete_scaled, y_train_IQR_delete)
print(f'IQR_삭제 모델 : {time()-start}')

# 검증 데이터를 이용한 예측
y_val_pred = mlp_IQR_delete.predict(X_val_IQR_delete_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

IQR_삭제 모델 : 5.823083400726318
              precision    recall  f1-score   support

           0       0.55      0.72      0.63      7276
           1       0.56      0.35      0.43      5052
           2       0.44      0.41      0.42      4553

    accuracy                           0.53     16881
   macro avg       0.52      0.49      0.49     16881
weighted avg       0.52      0.53      0.51     16881

[[5255  786 1235]
 [2136 1758 1158]
 [2082  612 1859]]
0.5255612819145785
0.5255612819145785


___

# 3. IQR_가중치 데이터셋
- 울타리 바깥 범위의 데이터에 대해 거리의 역수를 가중치로 적용. 해당 데이터셋으로 학습 후 확인

In [10]:
IQR_weight = pd.read_csv('../data/IQR_weight_encoding.csv', index_col=0)
X_train_IQR_weight, y_train_IQR_weight = IQR_weight.iloc[:,:-1], IQR_weight.iloc[:,-1]

scaler = StandardScaler()
X_train_IQR_weight, y_train_IQR_weight = IQR_weight.iloc[:,:-1], IQR_weight.iloc[:,-1]
X_train_IQR_weight_scaled = scaler.fit_transform(X_train_IQR_weight)
X_val_IQR_weight_scaled = scaler.transform(X_val)
X_test_IQR_weight_scaled = scaler.transform(X_test)

In [11]:
start = time()

mlp = MLPClassifier(solver='adam', activation='relu')

# 적절한 hidden_layer_size와 max_iter값을 위한 과정
# 은닉층 개수는 2개까지, 은닉층의 노드 개수는 피처 개수 이하로 설정.
# params = {
#     'hidden_layer_sizes' : Categorical([(10,), (20,), (10,10), (20,20)]),
#     'alpha' : Real(0.0001, 0.05, 'uniform'),
#     'learning_rate' : Categorical(['constant']),
#     'max_iter' : Integer(1000,2000)
# }
params = {
    'hidden_layer_sizes' : [(10,), (20,)],
    'alpha' : uniform(loc=0.001, scale=0.01),
    'learning_rate' : ['constant'],
    'max_iter' : range(1000,2000)
}
random_search = RandomizedSearchCV(mlp, params, cv = 5) # 5겹 교차 검증 보통 사용하는 값으로 10도 있음.
random_search.fit(X_train_IQR_weight_scaled, y_train_IQR_weight)
print(f'IQR_가중치 모델 파라미터 찾기 : {time()-start}')
params_IQR_weight = random_search.best_params_
print(params_IQR_weight)

IQR_가중치 모델 파라미터 찾기 : 491.68941259384155
{'alpha': 0.010293901422908563, 'hidden_layer_sizes': (10,), 'learning_rate': 'constant', 'max_iter': 1962}


In [12]:
# mlp 모델 학습
start = time()

mlp_IQR_weight = MLPClassifier(hidden_layer_sizes = params_IQR_weight['hidden_layer_sizes'],
                         alpha = params_IQR_weight['alpha'],
                         learning_rate = params_IQR_weight['learning_rate'],
                         max_iter = params_IQR_weight['max_iter'])
mlp_IQR_weight.fit(X_train_IQR_weight_scaled, y_train_IQR_weight)
print(f'IQR_가중치 모델 : {time()-start}')

# 검증 데이터를 이용한 예측
y_val_pred = mlp_IQR_weight.predict(X_val_IQR_weight_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

IQR_가중치 모델 : 14.81403636932373
              precision    recall  f1-score   support

           0       0.53      0.76      0.62      7276
           1       0.56      0.35      0.43      5052
           2       0.44      0.32      0.37      4553

    accuracy                           0.52     16881
   macro avg       0.51      0.48      0.47     16881
weighted avg       0.51      0.52      0.50     16881

[[5546  780  950]
 [2452 1747  853]
 [2508  600 1445]]
0.5176233635448136
0.5176233635448136


___

# 4. Isolation Forest_삭제 데이터셋
- Isolation Forest로 이상치 탐지 후 이상치로 판단된 관측치는 삭제한 데이터셋으로 학습 후 확인

In [13]:
Isolation_Forest_delete = pd.read_csv('../data/Isolation_Forest_delete_encoding.csv', index_col=0)
X_train_Isolation_Forest_delete, y_train_Isolation_Forest_delete = Isolation_Forest_delete.iloc[:,:-1], Isolation_Forest_delete.iloc[:,-1]

scaler = StandardScaler()
X_train_Isolation_Forest_delete, y_train_Isolation_Forest_delete = Isolation_Forest_delete.iloc[:,:-1], Isolation_Forest_delete.iloc[:,-1]
X_train_Isolation_Forest_delete_scaled = scaler.fit_transform(X_train_Isolation_Forest_delete)
X_val_Isolation_Forest_delete_scaled = scaler.transform(X_val)
X_test_Isolation_Forest_delete_scaled = scaler.transform(X_test)

In [14]:
start = time()

mlp = MLPClassifier(solver='adam', activation='relu')

# 적절한 hidden_layer_size와 max_iter값을 위한 과정
# 은닉층 개수는 2개까지, 은닉층의 노드 개수는 피처 개수 이하로 설정.
# params = {
#     'hidden_layer_sizes' : Categorical([(10,), (20,), (10,10), (20,20)]),
#     'alpha' : Real(0.0001, 0.05, 'uniform'),
#     'learning_rate' : Categorical(['constant']),
#     'max_iter' : Integer(1000,2000)
# }
params = {
    'hidden_layer_sizes' : [(10,), (20,)],
    'alpha' : uniform(loc=0.001, scale=0.01),
    'learning_rate' : ['constant'],
    'max_iter' : range(1000,2000)
}
random_search = RandomizedSearchCV(mlp, params, cv = 5) # 5겹 교차 검증 보통 사용하는 값으로 10도 있음.
random_search.fit(X_train_Isolation_Forest_delete_scaled, y_train_Isolation_Forest_delete)
print(f'Isolation Forest_삭제 모델 파라미터 찾기 : {time()-start}')
params_Isolation_Forest_delete = random_search.best_params_
print(params_Isolation_Forest_delete)

Isolation Forest_삭제 모델 파라미터 찾기 : 434.37979340553284
{'alpha': 0.008961785624242047, 'hidden_layer_sizes': (10,), 'learning_rate': 'constant', 'max_iter': 1289}


In [15]:
# mlp 모델 학습
start = time()

mlp_Isolation_Forest_delete = MLPClassifier(hidden_layer_sizes = params_Isolation_Forest_delete['hidden_layer_sizes'],
                         alpha = params_Isolation_Forest_delete['alpha'],
                         learning_rate = params_Isolation_Forest_delete['learning_rate'],
                         max_iter = params_Isolation_Forest_delete['max_iter'])
mlp_Isolation_Forest_delete.fit(X_train_Isolation_Forest_delete_scaled, y_train_Isolation_Forest_delete)
print(f'Isolation Forest_삭제 모델 : {time()-start}')
# 검증 데이터를 이용한 예측
y_val_pred = mlp_Isolation_Forest_delete.predict(X_val_Isolation_Forest_delete_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

Isolation Forest_삭제 모델 : 5.995054244995117
              precision    recall  f1-score   support

           0       0.54      0.74      0.63      7276
           1       0.56      0.34      0.43      5052
           2       0.43      0.37      0.40      4553

    accuracy                           0.52     16881
   macro avg       0.51      0.48      0.48     16881
weighted avg       0.52      0.52      0.51     16881

[[5403  741 1132]
 [2257 1727 1068]
 [2275  602 1676]]
0.5216515609264853
0.5216515609264853


___

# 5. Isolation Forest_가중치 데이터셋
- Isolation Forest로 이상치 탐지 후 이상치로 판단된 관측치에는 MinMaxScale을 진행한 이상치 점수를 가중치로 적용. 해당 데이터셋으로 학습 후 확인

In [16]:
Isolation_Forest_weight = pd.read_csv('../data/Isolation_Forest_weight_encoding.csv', index_col=0)
X_train_Isolation_Forest_weight, y_train_Isolation_Forest_weight = Isolation_Forest_weight.iloc[:,:-1], Isolation_Forest_weight.iloc[:,-1]

scaler = StandardScaler()
X_train_Isolation_Forest_weight, y_train_Isolation_Forest_weight = Isolation_Forest_weight.iloc[:,:-1], Isolation_Forest_weight.iloc[:,-1]
X_train_Isolation_Forest_weight_scaled = scaler.fit_transform(X_train_Isolation_Forest_weight)
X_val_Isolation_Forest_weight_scaled = scaler.transform(X_val)
X_test_Isolation_Forest_weight_scaled = scaler.transform(X_test)

In [17]:
start = time()

mlp = MLPClassifier(solver='adam', activation='relu')

# 적절한 hidden_layer_size와 max_iter값을 위한 과정
# 은닉층 개수는 2개까지, 은닉층의 노드 개수는 피처 개수 이하로 설정.
# params = {
#     'hidden_layer_sizes' : Categorical([(10,), (20,), (10,10), (20,20)]),
#     'alpha' : Real(0.0001, 0.05, 'uniform'),
#     'learning_rate' : Categorical(['constant']),
#     'max_iter' : Integer(1000,2000)
# }
params = {
    'hidden_layer_sizes' : [(10,), (20,)],
    'alpha' : uniform(loc=0.001, scale=0.01),
    'learning_rate' : ['constant'],
    'max_iter' : range(1000,2000)
}
random_search = RandomizedSearchCV(mlp, params, cv = 5) # 5겹 교차 검증 보통 사용하는 값으로 10도 있음.
random_search.fit(X_train_Isolation_Forest_weight_scaled, y_train_Isolation_Forest_weight)
print(f'Isolation Forest_가중치 모델 파라미터 찾기 : {time()-start}')
params_Isolation_Forest_weight = random_search.best_params_
print(params_Isolation_Forest_weight)

Isolation Forest_가중치 모델 파라미터 찾기 : 765.1102519035339
{'alpha': 0.0074115955942962415, 'hidden_layer_sizes': (20,), 'learning_rate': 'constant', 'max_iter': 1519}


In [18]:
# mlp 모델 학습
start = time()

mlp_Isolation_Forest_weight = MLPClassifier(hidden_layer_sizes = params_Isolation_Forest_weight['hidden_layer_sizes'],
                         alpha = params_Isolation_Forest_weight['alpha'],
                         learning_rate = params_Isolation_Forest_weight['learning_rate'],
                         max_iter = params_Isolation_Forest_weight['max_iter'])
mlp_Isolation_Forest_weight.fit(X_train_Isolation_Forest_weight_scaled, y_train_Isolation_Forest_weight)
print(f'Isolation Forest_가중치 모델 : {time()-start}')
# 검증 데이터를 이용한 예측
y_val_pred = mlp_Isolation_Forest_weight.predict(X_val_Isolation_Forest_weight_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

Isolation Forest_가중치 모델 : 23.706475973129272
              precision    recall  f1-score   support

           0       0.56      0.70      0.62      7276
           1       0.53      0.41      0.46      5052
           2       0.44      0.37      0.40      4553

    accuracy                           0.52     16881
   macro avg       0.51      0.49      0.49     16881
weighted avg       0.52      0.52      0.51     16881

[[5113 1025 1138]
 [1987 2062 1003]
 [2046  828 1679]]
0.5244949943723713
0.5244949943723713


___

데이터셋을 달리하여 만들어본 모델의 성능이 모두 차이가 없다고 봐도 무방할 정도이므로 베이스 모델을 강화할 것이다.